In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import keras
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
K.set_image_data_format('channels_last')

%matplotlib inline

/home/scanner-ml/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16492708217666113544
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2670198784
locality {
  bus_id: 1
}
incarnation: 6312969162566649163
physical_device_desc: "device: 0, name: GeForce GTX 780, pci bus id: 0000:03:00.0, compute capability: 3.5"
]


In [3]:
path = "/mnt/ML-drive/scanner-ml/Artem/share/Valerio/"
class_names = ['C100keV','TestSample']
n_pols = 8

In [4]:
def load_images(path, name_dir='TestSample', n_pols=8):
    
    img_ind = []
    path = path+name_dir+"/crops/"
    img_names = os.listdir(path)
    for name in img_names:
        img_ind.append(re.split('[_.]',name))
    img_ind = pd.DataFrame(np.array(img_ind),columns=['ViewID','gr','GrainID','pol','Polarization','cl','ClusterID','csv'])
    img_ind = img_ind.drop(['csv'],axis=1).sort_values(['ViewID','GrainID','Polarization'])
    for view in np.unique(img_ind['ViewID']):
        view_imgs = img_ind[ img_ind['ViewID']==view ]
        for grain in np.unique(view_imgs['GrainID']):
            grain_imgs = view_imgs[ view_imgs['GrainID']==grain ]
            if grain_imgs.shape[0]!=8:
                img_ind = img_ind.drop(grain_imgs.index)
    img_names = []
    for name in img_ind.values:
        img_names.append('_'.join(name)+'.csv')
    
    i=0
    im_array = []
    for name in img_names:
        if i==0 : tmp_im = []
        tmp_im.append(pd.read_csv(path+name, header=None).drop(31, axis=1).values)
        i+=1
        if i==n_pols:
            im_array.append(np.array(tmp_im).T)
            i=0
    return np.array(im_array)

In [5]:
%%time
X_sign = load_images(path, 'C100keV')
y_sign = np.ones((X_sign.shape[0],1))
X_noise = load_images(path, 'TestSample')
y_noise = np.zeros((X_noise.shape[0],1))

X = np.vstack((X_sign, X_noise))
y = np.vstack((y_sign, y_noise))

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.15, stratify=y)

# Normalize image vectors (????)
#X_train = X_train_orig/255.
#X_test = X_test_orig/255.

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 19703
number of test examples = 3478
X_train shape: (19703, 31, 31, 8)
Y_train shape: (19703, 1)
X_test shape: (3478, 31, 31, 8)
Y_test shape: (3478, 1)
CPU times: user 7min 32s, sys: 3.25 s, total: 7min 35s
Wall time: 7min 35s


# Sergey-like VGG model

In [17]:
def VGG_model(input_shape=(31,31,8), classes=2):
    
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((1,1))(X_input)
    X = Conv2D(64, 3, activation='relu', name='conv1_1')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Conv2D(64, 3, name='conv1_2')(X)
    X = BatchNormalization(name='batch1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(name='pool1')(X)
    print('conv1\t',X.get_shape())
    
    X = ZeroPadding2D((1,1))(X)
    X = Conv2D(128, 3, name='conv2')(X)
    X = BatchNormalization(name='batch2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(name='pool2')(X)
    print('conv2\t',X.get_shape())
    
    X = Dropout(rate=0.5, name='drop_middle')(X)
    
    X = ZeroPadding2D((1,1))(X)
    X = Conv2D(256, 3, name='conv3')(X)
    X = BatchNormalization(name='batch3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(name='pool3')(X)
    print('conv3\t',X.get_shape())
    
    X = ZeroPadding2D((1,1))(X)
    X = Conv2D(256, 3, name='conv4')(X)
    X = BatchNormalization(name='batch4')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(name='pool4')(X)
    print('conv4\t',X.get_shape())
    
    X = Dropout(rate=0.5, name='drop')(X)
    
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    if classes != 2 : print('oh no, too many classes, change the model output to softmax!')
    
    model = Model(inputs = X_input, outputs = X, name='VGG_model')
    return model

In [18]:
model = VGG_model(input_shape=(31,31,8))

conv1	 (?, 15, 15, 64)
conv2	 (?, 7, 7, 128)
conv3	 (?, 3, 3, 256)
conv4	 (?, 1, 1, 256)


In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, Y_train, epochs = 30, batch_size = 32, validation_split=0.15)

Train on 16747 samples, validate on 2956 samples
Epoch 1/30
16747/16747 [==============================] - 18s 1ms/step - loss: 0.1397 - acc: 0.9503 - val_loss: 1.2302 - val_acc: 0.6979
Epoch 2/30
16747/16747 [==============================] - 18s 1ms/step - loss: 0.1003 - acc: 0.9639 - val_loss: 1.0896 - val_acc: 0.6976
Epoch 3/30
16747/16747 [==============================] - 18s 1ms/step - loss: 0.0852 - acc: 0.9698 - val_loss: 0.9155 - val_acc: 0.3914
Epoch 4/30
16747/16747 [==============================] - 18s 1ms/step - loss: 0.0763 - acc: 0.9737 - val_loss: 1.2293 - val_acc: 0.3058
Epoch 5/30
16747/16747 [==============================] - 18s 1ms/step - loss: 0.0720 - acc: 0.9758 - val_loss: 1.3417 - val_acc: 0.1722
Epoch 6/30
16747/16747 [==============================] - 18s 1ms/step - loss: 0.0625 - acc: 0.9784 - val_loss: 11.2105 - val_acc: 0.3041
Epoch 7/30
16747/16747 [==============================] - 18s 1ms/step - loss: 0.0596 - acc: 0.9803 - val_loss: 1.6595 - val_acc

In [22]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

3478/3478 [==============================] - 1s 306us/step
Loss = 4.460614165470064
Test Accuracy = 0.31196089715296815


# Simplified CNN model

In [30]:
def CNN_model(input_shape=(31,31,8), classes=2):
    
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((1,1))(X_input)
    X = Conv2D(64, 3, name='conv1')(X)
    #X = BatchNormalization(name='batch1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(name='pool1')(X)
    print('conv1\t',X.get_shape())
    
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    if classes != 2 : print('oh no, too many classes, change the model output to softmax!')
    
    model = Model(inputs = X_input, outputs = X, name='CNN_model')
    return model

In [31]:
cnn_model = CNN_model(input_shape=(31,31,8))

conv1	 (?, 15, 15, 64)


In [36]:
cnn_model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
cnn_model.fit(X_train, Y_train, epochs = 30, batch_size = 64, validation_split=0.15)

Train on 16747 samples, validate on 2956 samples
Epoch 1/30
16747/16747 [==============================] - 4s 230us/step - loss: 11.0806 - acc: 0.3125 - val_loss: 11.2216 - val_acc: 0.3038
Epoch 2/30
16747/16747 [==============================] - 3s 204us/step - loss: 11.0806 - acc: 0.3125 - val_loss: 11.2216 - val_acc: 0.3038
Epoch 3/30
16747/16747 [==============================] - 3s 203us/step - loss: 11.0806 - acc: 0.3125 - val_loss: 11.2216 - val_acc: 0.3038
Epoch 4/30
16747/16747 [==============================] - 3s 201us/step - loss: 11.0806 - acc: 0.3125 - val_loss: 11.2216 - val_acc: 0.3038
Epoch 5/30
16747/16747 [==============================] - 3s 198us/step - loss: 11.0806 - acc: 0.3125 - val_loss: 11.2216 - val_acc: 0.3038
Epoch 6/30
16747/16747 [==============================] - 3s 201us/step - loss: 11.0806 - acc: 0.3125 - val_loss: 11.2216 - val_acc: 0.3038
Epoch 7/30
16747/16747 [==============================] - 3s 195us/step - loss: 11.0806 - acc: 0.3125 - val_los

In [38]:
preds = cnn_model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

3478/3478 [==============================] - 0s 131us/step
Loss = 11.099148458554321
Test Accuracy = 0.3113858540247335
